In [ ]:
%pip install python-dotenv kaggle

import os
from dotenv import load_dotenv

load_dotenv()

if not os.getenv("KAGGLE_USERNAME") or not os.getenv("KAGGLE_KEY"):
    print("Error: Kaggle credentials not found in .env file.")
else:
    print("Credentials loaded successfully.")

!kaggle datasets download -d birdy654/cifake-real-and-ai-generated-synthetic-images

print("Downloading and unzipping dataset...")

!unzip -q cifake-real-and-ai-generated-synthetic-images.zip

print("Done")

Unzipping dataset...


'kaggle' is not recognized as an internal or external command,
operable program or batch file.


Done


'unzip' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import os

if os.path.exists('train') and os.path.exists('test'):
    print(f"Real Images Dataset: {len(os.listdir('train/REAL'))}")
    print(f"Fake Images Dataset: {len(os.listdir('train/FAKE'))}")
else:
    print("Error, datasets not found")

Something went wrong.


In [ ]:
import os
import cv2
import joblib
import numpy as np
from tqdm import tqdm
from pathlib import Path
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

MAX_IMAGES = 2000 
IMG_SIZE = 128

MODEL_PATH = Path("../model/")

def get_azimuthal_average(magnitude_spectrum):
    h, w = magnitude_spectrum.shape
    center = (h // 2, w // 2)
    y, x = np.indices((h, w))
    
    r = np.sqrt((x - center[0])**2 + (y - center[1])**2).astype(int)
    
    tbin = np.bincount(r.ravel(), weights=magnitude_spectrum.ravel())
    nr = np.bincount(r.ravel())
    
    radial_profile = tbin / np.maximum(nr, 1)
    
    return radial_profile[:60] 

def extract_features(img_path):
    try:
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        
        if img is None: return None
        
        img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
        
        f = np.fft.fft2(img)
        fshift = np.fft.fftshift(f)
        magnitude_spectrum = 20 * np.log(np.abs(fshift) + 1e-9)
        
        return get_azimuthal_average(magnitude_spectrum)
        
    except Exception as e:
        return None

def load_dataset(root_dir, subset_name):
    features = []
    labels = []
    
    classes = {'REAL': 0, 'FAKE': 1}
    
    base_path = os.path.join(root_dir, subset_name)
    
    for label_name, label_idx in classes.items():
        folder_path = os.path.join(base_path, label_name)
        image_files = os.listdir(folder_path)
        
        if MAX_IMAGES:
            image_files = image_files[:MAX_IMAGES]
            
        print(f"Processing {len(image_files)} images from {subset_name}/{label_name}...")
        
        for fname in tqdm(image_files):
            path = os.path.join(folder_path, fname)
            data = extract_features(path)
            
            if data is not None:
                features.append(data)
                labels.append(label_idx)
                
    return np.array(features), np.array(labels)

print("Loading Training Data...")

X_train, y_train = load_dataset('.', 'train')

print("\nLoading Test Data...")

X_test, y_test = load_dataset('.', 'test')

print("\nTraining SVM Classifier...")

model = SVC(kernel='rbf', probability=True) 
model.fit(X_train, y_train)

print("\nEvaluating Model...")

predictions = model.predict(X_test)
acc = accuracy_score(y_test, predictions)

print(f"Model Accuracy: {acc * 100:.2f}%")
print(classification_report(y_test, predictions, target_names=['REAL', 'FAKE']))

print("\nSaving Model...")

joblib.dump(model, f"{MODEL_PATH}/deepfake_detector.pkl")

print(f"Saved: {MODEL_PATH}/deepfake_detector.pkl")

--- Loading Training Data ---


FileNotFoundError: [WinError 3] The system cannot find the path specified: '.\\train\\REAL'